## Summary Tables
#### *Creating summary tables for three different queries of the Wedge data*

### Code imports & set GBQ connection

In [2]:
# Do our imports for the code
import os
import io
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
# set up connection to Google Big Query
service_path = "/Users/natebender/Desktop/Wedgekey/"
service_file = 'Bender-Wedge-79e456939915.json' # change this to your authentication information  
gbq_proj_id = 'bender-wedge' # change this to your project. 
gbq_dataset_id = 'wedgeclean' # and change this to your data set ID

private_key = service_path + service_file

In [4]:
# Identify path to the clean Wedge files and establish a list to store the files
clean_files = "/Users/natebender/Desktop/Repo/ada-wedge/data/"
clean_wedge_files = os.listdir(clean_files)

# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

### Connect to SQLite and create database for this project

In [8]:
# connect to sqlite and create the WedgeTask3 database file
import sqlite3
db = sqlite3.connect("NBwedgetask3.db")
cur = db.cursor()

### Extract sales by date by hour data from GBQ, input into database 
#### *Example use case: how have our per-day sales changed over the last few months?*

In [6]:
# First query of Wedge data in GBQ: Date, Hour, Sales, Transactions, and Items
query1 = (
    """SELECT (EXTRACT(date FROM datetime)) AS Date,
    (EXTRACT(hour FROM datetime)) AS Hour,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedgeclean.wedge-data-all` 
    WHERE card_no != 3
    AND department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY Date, Hour
    ORDER BY Date, Hour"""
)

# And we execute queries with `client.query`
results1 = client.query(
    query1,
    location="US",
)

In [7]:
# output the first query results as a text file
with open('Sales_by_Date_Hour.txt', 'w') as outfile :
    for line in results1 :
        row = [str(item) for item in line]
        outfile.write(",".join(row) + "\n")
    

In [9]:
# input the results of the first query into the NBwedgetask3 database as the table Sales_by_Date_Hour
input_file1 = "Sales_by_Date_Hour.txt"

db = sqlite3.connect("NBwedgetask3.db") # connect to the WedgeTask 3 database
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS Sales_by_Date_Hour''') # remove the table if it already exists

# create the table in the connected database and set up the schema
cur.execute('''CREATE TABLE Sales_by_Date_Hour (
    Date TIMESTAMP,
    Hour TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items TEXT)''')

# input the Sales by Date by Hour text data into the established table in the database
with open(input_file1, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Sales_by_Date_Hour (Date, Hour, Sales, Transactions, Items)
        VALUES (?,?,?,?,?)''', line)
db.commit()

### Extract sales by owner by year and month, input into database
#### *Example use case: What is our most popular item in each department?*
#### *This table has the following columns: card_no, year, month, sales, transactions, and items.*

In [10]:
# Second Query of Wedge data in GBQ - Owner, Year, Month, Sales, Transactions, and Items
query2 = (
    """SELECT card_no As Owner,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedgeclean.wedge-data-all` 
    WHERE card_no != 3
    AND department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY Owner, Year, Month
    ORDER BY Owner, Year, Month DESC"""
)

# And we execute queries with `client.query`
results2 = client.query(
    query2,
    location="US",
)

In [11]:
# output the second query results as a text file
with open('Sales_by_Owner_Date.txt', 'w') as outfile :
    for line in results2 :
        row = [str(item) for item in line]
        outfile.write(",".join(row) + "\n")
        

In [12]:
# input the results of the first query into the NBwedgetask3 database as the table Sales_by_Owner_Date
input_file2 = "Sales_by_Owner_Date.txt"

db = sqlite3.connect("NBwedgetask3.db") # connect to the WedgeTask 3 database
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS Sales_by_Owner_Date''') # remove the table if it already exists

# create the table in the connected database and set up the schema
cur.execute('''CREATE TABLE Sales_by_Owner_Date (
    Owner INTEGER,
    Year TIMESTAMP,
    Month TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items TEXT)''')

# input the Sales by Owner by Date text data into the established table in the database
with open(input_file2, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Sales_by_Owner_Date (Owner, Year, Month, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?)''', line)
db.commit()

### Extract sales by product description by year and month, input into database
#### *Example use case: which owners spend the most per month in each department?*
#### *This table has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.*

In [13]:
# Third Query of Wedge data in GBQ - Product, Date, Sales, Transactions, and Items
query3 = (
    """SELECT Upc AS UPC,
    description AS Product,
    a.department AS Department,
    b.dept_name AS Dept_Name,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedgeclean.wedge-data-all` a
    LEFT OUTER JOIN `wedgeclean.department_lookup` b
    ON a.department = b.department
    WHERE card_no != 3
    AND a.department != 0
    AND a.department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY UPC, Product, Department, Dept_Name, Year, Month
    ORDER BY UPC, Product, Department, Dept_Name, Year, Month DESC"""
)

# And we execute queries with `client.query`
results3 = client.query(
    query3,
    location="US",
)

In [14]:
# output the third query results as a text file
with open('Sales_by_Product_Date.txt', 'w') as outfile :
    for line in results3 :
        row = [str(item) for item in line]
        outfile.write(",".join(row) + "\n")

In [16]:
# input the results of the third query into the NBwedgetask3 database as the table Sales_by_Product_Date
input_file3 = "Sales_by_Product_Date.txt"

db = sqlite3.connect("NBwedgetask3.db")# connect to the WedgeTask 3 database
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS Sales_by_Product_Date''') # remove the table if it already exists

# create the table in the connected database and set up the schema
cur.execute('''CREATE TABLE Sales_by_Product_Date (
    UPC STRING,
    Description STRING,
    Department FLOAT,
    Dept_Name STRING,
    Year TIMESTAMP,
    Month TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items TEXT)''')

# input the Sales by Product by Date text data into the established table in the database
with open(input_file3, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Sales_by_Product_Date (UPC, Description, Department, Dept_Name, Year, Month, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?,?,?,?)''', line)
db.commit()